In [33]:

# coding: utf-8

# In[80]:


import pandas as pd
from sodapy import Socrata
import numpy as np 
import ast
import json

CommunityToPoliceDistrict = []
with open('PoliceDistrictToCommunityNamesOutput.txt') as fp:
    for line in fp:
        communityNum, communityName, policeDist  = line.strip().split(":")
        communityName = communityName.strip()
        CommunityToPoliceDistrict.append({"Community Area": float(communityNum), "Community_Name": communityName,
                                         "Police_District": policeDist})

CommunityPoliceDistrictDF = pd.DataFrame(CommunityToPoliceDistrict)
CommunityPoliceDistrictDF.head()

,Community Area,Community_Name,Police_District
0,1.0,ROGERS PARK,24
1,2.0,WEST RIDGE,24
2,3.0,UPTOWN,23
3,4.0,LINCOLN SQUARE,20
4,5.0,NORTH CENTER,19


In [34]:

# In[81]:


CrimeData = pd.read_csv("Crimes_2001_2020.csv",
                       dtype={'ID': int, 'Case Number' : str, 'Date' : str, 'Block' : str, 
                             'IUCR' : str, 'Primary Type' : str, 'Description' : str, 
                             'Location Description' : str, 'Arrest' : str, 'Domestic' : str, 'Beat' : str, 
                             'District' : float, 'Ward' : float, 'Community Area' : float, 'FBI Code' : str, 
                              'X Coordinate' : str, 'Y Coordinate' : str, 'Year' : int,'Updated On' : str,
                             'Latitude' : str, 'Longitude' : str, 'Location' : str})

CrimeData = CrimeData.drop(['X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Location'], axis = 1)
First100CrimeData = CrimeData[:100]
First100CrimeData = First100CrimeData.join(CommunityPoliceDistrictDF.set_index('Community Area'), on='Community Area')
print(First100CrimeData.head())
print(First100CrimeData.columns)
print(First100CrimeData['Community Area'].unique())

         ID Case Number                    Date                 Block  IUCR  \
0  11034701    JA366925  01/01/2001 11:00:00 AM       016XX E 86TH PL  1153   
1  11227287    JB147188  10/08/2017 03:00:00 AM    092XX S RACINE AVE   281   
2  11227583    JB147595  03/28/2017 02:00:00 PM       026XX W 79TH ST   620   
3  11227293    JB147230  09/09/2017 08:17:00 PM  060XX S EBERHART AVE   810   
4  11227634    JB147599  08/26/2017 10:00:00 AM   001XX W RANDOLPH ST   281   

          Primary Type                          Description  \
0   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
1  CRIM SEXUAL ASSAULT                       NON-AGGRAVATED   
2             BURGLARY                       UNLAWFUL ENTRY   
3                THEFT                            OVER $500   
4  CRIM SEXUAL ASSAULT                       NON-AGGRAVATED   

  Location Description Arrest Domestic  Beat  District  Ward  Community Area  \
0            RESIDENCE  FALSE    FALSE   412       4.0   8.0      

In [35]:
# In[82]:


# Used to Extract the data from the Socio Economic CSV File 
# To train our program what specific communities are better or worse off than average
import pandas as pd 
import numpy as np 

# We can use .describe() to find the benchmark levels of poverty per community. Compare community to the average and give weightage 
columns = [
'Community Area',
'community_area_name', 
'%_housing_crowded', 
'%_households_below_poverty', 
'%_age16_unemployed',
'%_age25_nohighschool',
'%_not_working_age',
'per_capita_income',
'hardship_index']

SocioeconomicData = pd.read_csv("SocioeconomicData.csv")
SocioeconomicData.columns = columns

First100CrimeData = First100CrimeData.join(SocioeconomicData.set_index('Community Area'), on='Community Area')

print(First100CrimeData.columns)

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On',
       'Community_Name', 'Police_District', 'community_area_name',
       '%_housing_crowded', '%_households_below_poverty', '%_age16_unemployed',
       '%_age25_nohighschool', '%_not_working_age', 'per_capita_income',
       'hardship_index'],
      dtype='object')


In [36]:
# In[83]:


import pandas as pd

census = pd.read_csv("census_data.csv", header=1)

CensusData = pd.DataFrame(census)
CensusData = CensusData[CensusData.columns[range(67)]]
CensusData = CensusData.rename(columns={"Geog": "Community Name", "GeogKey": "Community Area"})

#First100CrimeData = First100CrimeData.join(CensusData.set_index('Community Area'), on='Community Area')
First100CrimeData = First100CrimeData.merge(CensusData, on='Community Area', how='left')
First100CrimeData = First100CrimeData.drop(
    ['Arrest', 'Domestic', 'FBI Code', 'IUCR', 'Beat', 'Ward',
    'Case Number', 'Block', 'Primary Type', 'Description',
       'Location Description', 'District', 'Community Area', 'Year',
       'Updated On', 'community_area_name', 'Police_District','Not Hispanic or Latino, American Indian and Alaska Native alone',
    'Not Hispanic or Latino, Native Hawaiian and Other Pacific Islander alone', 'Not Hispanic or Latino, Some Other Race alone','Not Hispanic or Latino, Two or More Races'], axis = 1)

First100CrimeData = First100CrimeData.dropna()
First100CrimeData.rename(columns={'Not Hispanic or Latino, White alone' : '#_White',
       'Not Hispanic or Latino, Black or African American alone' : "#_African_American",
       'Not Hispanic or Latino, Asian alone' : "#_Asian",'Hispanic or Latino' : "#_Hispanic"})



,ID,Date,Community_Name,%_housing_crowded,%_households_below_poverty,%_age16_unemployed,%_age25_nohighschool,%_not_working_age,per_capita_income,hardship_index,...,Median Age,Total Households,Average Household Size,Total Housing Units,Occupied Housing Units,Vacant Housing Units,Occupied Housing Units.1,Owned with a mortgage or a loan,Owned free and clear,Renter occupied
0,11034701,01/01/2001 11:00:00 AM,AVALON PARK,1.4,17.2,21.1,10.6,39.3,24454,41.0,...,42.9,3924.0,2.57,4295.0,3924.0,371.0,3924.0,1990.0,789.0,1145.0
1,11227287,10/08/2017 03:00:00 AM,WASHINGTON HEIGHTS,1.1,16.9,20.8,13.7,42.6,19713,48.0,...,41.2,9482.0,2.77,10365.0,9482.0,883.0,9482.0,4626.0,1921.0,2935.0
2,11227583,03/28/2017 02:00:00 PM,ASHBURN,4.0,10.4,11.7,17.7,36.9,23482,37.0,...,33.5,12451.0,3.25,13060.0,12451.0,609.0,12451.0,8900.0,1879.0,1672.0
3,11227293,09/09/2017 08:17:00 PM,WOODLAWN,2.9,30.7,23.4,16.5,36.1,18672,58.0,...,31.4,9400.0,2.37,12114.0,9400.0,2714.0,9400.0,1718.0,410.0,7272.0
4,11227634,08/26/2017 10:00:00 AM,LOOP,1.5,14.7,5.7,3.1,13.5,65526,3.0,...,30.5,15494.0,1.60,20557.0,15494.0,5063.0,15494.0,5147.0,1222.0,9125.0
5,11227517,02/10/2013 12:00:00 AM,GREATER GRAND CROSSING,3.6,29.6,23.0,16.5,41.0,17285,66.0,...,33.9,12585.0,2.55,15517.0,12585.0,2932.0,12585.0,2939.0,1277.0,8369.0
6,11227503,01/01/2015 12:01:00 AM,WEST LAWN,5.8,14.9,9.6,33.6,39.6,16907,56.0,...,30.0,8766.0,3.80,9362.0,8766.0,596.0,8766.0,5135.0,1446.0,2185.0
7,11227508,01/01/2017 12:01:00 AM,SOUTH LAWNDALE,15.2,30.7,15.8,54.8,33.8,10402,96.0,...,27.6,18131.0,3.79,21108.0,18131.0,2977.0,18131.0,4168.0,1792.0,12171.0
8,11022695,07/17/2017 10:10:00 AM,LOGAN SQUARE,3.2,16.8,8.2,14.8,26.2,31908,23.0,...,30.7,29352.0,2.46,32486.0,29352.0,3134.0,29352.0,8694.0,1504.0,19154.0
9,11227633,12/28/2017 03:55:00 PM,LOOP,1.5,14.7,5.7,3.1,13.5,65526,3.0,...,30.5,15494.0,1.60,20557.0,15494.0,5063.0,15494.0,5147.0,1222.0,9125.0


In [37]:
# In[84]:


from datetime import datetime, time
morningStart = time(6, 0, 0)
morningEnd = time(12,0,0)
afternoonStart = time(12,0,1)
afternoonEnd = time(18,30,0)
nightStart = time(18,31,0)
nightEnd = time(23,59,59)
midnightStart = time(0,0,0)
midnightEnd = time(5,59,0)

# Time of Day Function
def timeOfDayApplier(dateColumn):
    DatetimeVersion = datetime.strptime(dateColumn,'%m/%d/%Y %I:%M:%S %p') # 'I' accounts for 12-hour format
    
    if morningStart <= DatetimeVersion.time() <= morningEnd:
        return "Morning"
    elif afternoonStart <= DatetimeVersion.time() <= afternoonEnd:
        return "Afternoon/Evening"
    elif nightStart <= DatetimeVersion.time() <= nightEnd:
        return "Night"
    elif midnightStart <= DatetimeVersion.time() <= midnightEnd:
        return "Midnight"

First100CrimeData['TimeOfDay'] = First100CrimeData['Date'].map(timeOfDayApplier)


In [38]:
# In[85]:


from datetime import datetime, time
springStart = 3
springEnd = 5
summerStart = 6
summerEnd = 8
fallStart = 9
fallEnd = 11
winterStart = 12
winterEnd = 2 
# Time of Day Function
def seasonOfYear(dateColumn):
    DatetimeVersion = datetime.strptime(dateColumn,'%m/%d/%Y %I:%M:%S %p') # 'I' accounts for 12-hour format
     
    if DatetimeVersion.month in [12,1,2]:
        return "Winter"   
    elif springStart <= DatetimeVersion.month <= springEnd:
        return "Spring"
    elif summerStart <= DatetimeVersion.month <= summerEnd:
        return "Summer"
    elif fallStart <= DatetimeVersion.month <= fallEnd:
        return "Fall"

First100CrimeData['Season'] = First100CrimeData['Date'].map(seasonOfYear)

# Time of Day Function
def dayOfWeek(dateColumn):
    DatetimeVersion = datetime.strptime(dateColumn,'%m/%d/%Y %I:%M:%S %p') # 'I' accounts for 12-hour format
     
    if DatetimeVersion.weekday() in [0,1,2,3,4]:
        return "Weekday"   
    else:
        return "Weekend"

First100CrimeData['Weekday/Weekend'] = First100CrimeData['Date'].map(dayOfWeek)


# In[86]:


print(First100CrimeData['Date'][0]) # value check 
print(First100CrimeData['TimeOfDay'][0]) # value check
print(First100CrimeData['Season'][0]) # value check
print(First100CrimeData['Weekday/Weekend'][0]) # value check


01/01/2001 11:00:00 AM
Morning
Winter
Weekday


In [39]:
# In[87]:


print("Columns: ", First100CrimeData.columns)
print(First100CrimeData.head())
print("Shape: ", First100CrimeData.shape)

Columns:  Index(['ID', 'Date', 'Community_Name', '%_housing_crowded',
       '%_households_below_poverty', '%_age16_unemployed',
       '%_age25_nohighschool', '%_not_working_age', 'per_capita_income',
       'hardship_index', 'Community Name', 'Total Population',
       'Not Hispanic or Latino, White alone',
       'Not Hispanic or Latino, Black or African American alone',
       'Not Hispanic or Latino, Asian alone', 'Hispanic or Latino',
       'Male: Under 5 years old', 'Male: 5 to 9 years', 'Male: 10 to 14 years',
       'Male: 15 to 17 years', 'Male: 18 and 19 years', 'Male: 20 years',
       'Male: 21 years', 'Male: 22 to 24 years', 'Male: 25 to 29 years',
       'Male: 30 to 34 years', 'Male: 35 to 39 years', 'Male: 40 to 44 years',
       'Male: 45 to 49 years', 'Male: 50 to 54 years', 'Male: 55 to 59 years',
       'Male: 60 and 61 years', 'Male: 62 to 64 years',
       'Male: 65 and 66 years', 'Male: 67 to 69 years', 'Male: 70 to 74 years',
       'Male: 75 to 79 years', 'Ma

In [41]:
import pandasql
query = """
WITH cr AS
(
    SELECT DISTINCT
    Community_Name
    , COUNT(*) AS crimes 
    FROM First100CrimeData
    GROUP BY Community_Name
)
SELECT 
cr.crimes as Num_Crimes
, cr.Community_Name
FROM cr
ORDER BY Num_Crimes DESC, cr.Community_Name;
"""
CommunityNumCrimesDF = pandasql.sqldf(query, globals())
print(CommunityNumCrimesDF.head())
print("# CRIMES TOTAL: ",sum(CommunityNumCrimesDF.Num_Crimes))

   Num_Crimes          Community_Name
0           5  GREATER GRAND CROSSING
1           5                    LOOP
2           5         NEAR NORTH SIDE
3           3                 ASHBURN
4           3          AUBURN GRESHAM
# CRIMES TOTAL:  97
